Code modified from https://github.com/Vaibhavs10/fast-whisper-finetuning

# Prepare environment

In [1]:
%pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes==0.37 accelerate
%pip install -q git+https://github.com/huggingface/peft.git@main

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import notebook_login

notebook_login() #huggingface-cli login workaround

# Load Dataset

In [7]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

language_abbr = "hi" # Replace with the language ID of your choice here!

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_13_0", language_abbr, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_13_0", language_abbr, split="test", use_auth_token=True)

print(common_voice)

/Applications/miniconda3/envs/local_nmt/lib/python3.8/site-packages/datasets/load.py:1454: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Reading metadata...: 4479it [00:00, 144181.61it/s]les/s]
Generating train split: 4479 examples [00:00, 11160.58 examples/s]
Reading metadata...: 2281it [00:00, 223147.07it/s]examples/s]
Generating validation split: 2281 examples [00:00, 10969.04 examples/s]
Reading metadata...: 2947it [00:00, 240553.75it/s]es/s]
Generating test split: 2947 examples [00:00, 10995.52 examples/s]
Reading metadata...: 3487it [00:00, 249229.56it/s]les/s]
Gener

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 6760
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 2947
    })
})


In [8]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]
)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 6760
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2947
    })
})


In [9]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json: 100%|██████████| 185k/185k [00:00<00:00, 3.77MB/s]


In [10]:
from transformers import WhisperTokenizer

task = "transcribe"

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language=language_abbr, task=task)

tokenizer_config.json: 100%|██████████| 805/805 [00:00<00:00, 529kB/s]
vocab.json: 100%|██████████| 836k/836k [00:00<00:00, 12.6MB/s]
tokenizer.json: 100%|██████████| 2.48M/2.48M [00:00<00:00, 16.0MB/s]
merges.txt: 100%|██████████| 494k/494k [00:00<00:00, 6.23MB/s]
normalizer.json: 100%|██████████| 52.7k/52.7k [00:00<00:00, 31.0MB/s]
added_tokens.json: 100%|██████████| 34.6k/34.6k [00:00<00:00, 17.7MB/s]
special_tokens_map.json: 100%|██████████| 2.08k/2.08k [00:00<00:00, 2.41MB/s]


In [11]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language=language_abbr, task=task)

# Prepare data

In [12]:
print(common_voice["train"][0])

{'audio': {'path': '/Users/amyzhou/.cache/huggingface/datasets/downloads/extracted/434586742ec36ad34a876813d2a17bf0b9e979d6cb4f14cd22e11cf8cab61fdf/hi_train_0/common_voice_hi_26008353.mp3', 'array': array([ 1.17937861e-25, -6.46234854e-27, -1.37324906e-26, ...,
        1.06425851e-07,  4.46414674e-08,  2.61410094e-09]), 'sampling_rate': 48000}, 'sentence': 'हमने उसका जन्मदिन मनाया।'}


In [13]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [14]:
print(common_voice["train"][0])

{'audio': {'path': '/Users/amyzhou/.cache/huggingface/datasets/downloads/extracted/434586742ec36ad34a876813d2a17bf0b9e979d6cb4f14cd22e11cf8cab61fdf/hi_train_0/common_voice_hi_26008353.mp3', 'array': array([-6.93889390e-18, -1.90819582e-17, -3.46944695e-17, ...,
       -1.31141860e-07,  2.61934474e-07,  4.65079211e-08]), 'sampling_rate': 16000}, 'sentence': 'हमने उसका जन्मदिन मनाया।'}


In [15]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [16]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2): 100%|██████████| 2947/2947 [00:17<00:00, 166.80 examples/s]


In [17]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 6760
})

# Training and evaluation

In [18]:
# Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [19]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [20]:
# Evaluation metrics: during evaluation, we want to evaluate the model using the word error rate (WER) metric.
import evaluate

metric = evaluate.load("wer")

In [22]:
# Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small", load_in_8bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
# Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.
